In [9]:
import json
from gurobipy import Model, GRB, quicksum

# Compatibility function
def compatibility (dbtype, rbtype):
    comp_matrix = {
        "A": ["A", "AB"],
        "B": ["B", "AB"],
        "AB": ["AB"],
        "O": ["A", "B", "AB", "O"]
    }
    return rbtype in comp_matrix.get(dbtype, [])

# Dataset - allows for different datasets to be used if needed
def load_data(file):
    with open(file, 'r') as file:
        data = json.load(file)
    return data

# Create a compatibility graph
def Comp_graph(data):
    edges = []
    for i, recipient1 in enumerate(data):
        for j, recipient2 in enumerate(data):
            if i != j:
                #compatibility check
                for donor1 in recipient1["Donor"]:
                    for donor2 in recipient2["Donor"]:
                        if compatibility(donor1, recipient2["Recipient"]) and \
                           compatibility(donor2, recipient1["Recipient"]):
                            edges.append((i, j))
                            break
    return edges

# Optimization model
def kpd(file):
    data = load_data(file)
    edges = Comp_graph(data)

    model = Model("KPD Matching")

    x = model.addVars(edges, vtype=GRB.BINARY, name="x")

    model.setObjective(quicksum(x[e] for e in edges), GRB.MAXIMIZE)

    # Constraints
    for i in range(len(data)):
        model.addConstr(quicksum(x[e] for e in edges if e[0] == i or e[1] == i) <= 1, name=f"node_{i}_constraint")

    model.optimize()

    if model.status == GRB.OPTIMAL:
        print("Optimal solution found!")
        for e in edges:
            if x[e].x > 0.5:
                print(f"Matching: Pair {e[0]} (Recipient: {data[e[0]]['Recipient']}) ↔ "
                      f"Pair {e[1]} (Recipient: {data[e[1]]['Recipient']})")
    else:
        print("No optimal solution found.")

        
file = "To_Be_Determined.json"  
kpd(file)


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 741 rows, 76254 columns and 152508 nonzeros
Model fingerprint: 0xade9496f
Variable types: 0 continuous, 76254 integer (76254 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 179.0000000
Presolve removed 349 rows and 38127 columns
Presolve time: 0.35s
Presolved: 392 rows, 38127 columns, 76254 nonzeros
Found heuristic solution: objective 179.0000000
Variable types: 0 continuous, 38127 integer (38127 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity     -3.9100000e+0